# Modelos Alternativos

**Proyecto Integrador TC5035.10**

**Profesor Asesor:**
- Dra. Ludivina Facundo (ITESM)


**Profesores Investigadores/Tutores:**
- Dr. Juan Arturo Nolazco (ITESM)
- Dr. Marcos Faunez Zaunuy (TecnoCampus Barcelona)

**Equipo 11:**
- Francisco José Arellano Montes (A01794283)
- Armando Bringas Corpus (A01200230)
- Moisés Díaz Malagón (A01208580)

En esta entrega utilizaremos estrategias de ensamble homogéneas y heterogéneas, stacking y blending, para los modelos individuales de mejor rendimiento obtenidos en la fase anterior. Se incluirán también la optimización de hiperparámetros para los modelos más relevantes.

In [102]:
import warnings

import matplotlib.pyplot as plt
import seaborn as sns
import graphviz

import numpy as np
import pandas as pd
import time

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchview import draw_graph

from sklearn.metrics import (classification_report, roc_auc_score, precision_recall_curve,
                             auc, roc_curve, det_curve)
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split



from lightgbm import LGBMClassifier
from xgboost import XGBClassifier


from utils import load_data, print_classification_report, plot_roc_det_curve

In [103]:
# Carga de datos
X_train, y_train, X_val, y_val, X_text, y_test = load_data()

assert X_train.shape[1] == X_val.shape[1], "Las dimensiones de las características no coinciden"
assert y_train.shape[0] == X_train.shape[0], "Las dimensiones de las etiquetas no coinciden"

X_train.shape, y_train.shape, X_val.shape, y_val.shape

((571, 12), (571, 3), (556, 12), (556, 3))

In [104]:
# Importamos los mejores modelos previos

# 1. XGBoost
best_xgb_parameters = {'learning_rate': 0.01,
                        'max_depth': 3,
                        'n_estimators': 50,
                        'scale_pos_weight': 2.7320261437908497
                    }
xgb_model = XGBClassifier(**best_xgb_parameters, random_state=42)
xgb_model.fit(X_train, y_train['depression'])

# 2. SVM

svm_best_hyperparameters = {'C': 10, 'gamma': 'scale', 'kernel': 'linear'}

svm_model = SVC(**svm_best_hyperparameters, probability=True, class_weight='balanced', random_state=42)
svm_model.fit(X_train, y_train['depression'])

# 3. LightGBM

lgbm_model = LGBMClassifier(class_weight='balanced', random_state=42, verbose=-1)
lgbm_model.fit(X_train, y_train['depression'])


LGBMClassifier(class_weight='balanced', random_state=42, verbose=-1)

# Modelos de ensamble

### 1.1 Ensamble 1: Homogéneo de XGBoost usando stacking

Blending es una técnica de ensamble similar a Stacking, pero más simple. En lugar de entrenar un meta-modelo con predicciones de modelos base en un conjunto separado, Blending usa un pequeño conjunto de validación para generar predicciones y luego entrena un modelo final sobre estas predicciones.

In [105]:
xgboost_1 = XGBClassifier(
                            learning_rate=0.01,
                            max_depth=3,
                            n_estimators=50,
                            scale_pos_weight=2.7320261437908497, 
                            random_state=42
                        )
xgboost_2 = XGBClassifier(
                            learning_rate=0.1,
                            max_depth=5,
                            n_estimators=100,
                            scale_pos_weight=2.7320261437908497, 
                            random_state=42
                        )

xgboost_3 = XGBClassifier(
                            learning_rate=0.1,
                            max_depth=10,
                            n_estimators=50,
                            scale_pos_weight=2.7320261437908497, 
                            random_state=42
                        )

estimators = [
    ("XGB1", xgboost_1),
    ("XGB2", xgboost_2),
    ("XGB3", xgboost_3),
]

ensemble1 = StackingClassifier(estimators=estimators, final_estimator=XGBClassifier())
ensemble1

StackingClassifier(estimators=[('XGB1',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None,
                                              feature_types=None, gamma=None,
                                              grow_policy=None,
                                              importance_type=None,
                                              interaction_constraints=None,
                                              learning...
                                                 grow_policy=None,
                                                 importance_type=None,
                                                 interaction_constraints=None,
                                                 learning_rate=None,
                                                 max_bin=None,
                                                 max_cat_threshold=None,
                                                 max_cat_to_onehot=None,
                                                 max_delta_step=None,
                                                 max_depth=None,
                                                 max_leaves=None,
                                                 min_child_weight=None,
                                                 missing=nan,
                                                 monotone_constraints=None,
                                                 multi_strategy=None,
                                                 n_estimators=None, n_jobs=None,
                                                 num_parallel_tree=None,
                                                 random_state=None, ...))

In [106]:
start_time = time.time()
ensemble1.fit(X_train, y_train['depression'])
ensemble1_training_time = time.time() - start_time

y_pred_ensemble1 = ensemble1.predict(X_val)
y_prob_ensemble1 = ensemble1.predict_proba(X_val)[:, 1]
print_classification_report(y_val['stress'], y_pred_ensemble1, y_prob_ensemble1, label="Ensemble 1")
print("Elapsed trainint time: ", ensemble1_training_time, "s")

Ensemble 1 Classification Report:
              precision    recall  f1-score   support

       False       0.61      0.29      0.39       315
        True       0.45      0.76      0.56       241

    accuracy                           0.49       556
   macro avg       0.53      0.52      0.48       556
weighted avg       0.54      0.49      0.47       556

Ensemble 1 ROC AUC: 0.5224
Ensemble 1 Precision-Recall AUC: 0.4856
Elapsed trainint time:  0.45076799392700195 s


### 1.2 Ensamble 2: Homogéneo de SVM usando blending

In [107]:
class BlendingClassifier():
    def __init__(self, models, meta_model):
        self.models = models
        self.meta_model = meta_model
        self.training_time = None


    def fit(self, X_train, y_train):
        """
        Entrena un modelo de Blending con los modelos dados
        """

        X_train, X_holdout, y_train, y_holdout = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
        X_holdout_train, X_holdout_test, y_holdout_train, y_holdout_test = train_test_split(
            X_holdout, y_holdout, test_size=0.5, random_state=42
        )

        start_time = time.time()
        # Entrenamos modelos base con el conjunto de entrenamiento
        for model in self.models:
            model.fit(X_train, y_train)

        training_time = time.time() - start_time

        # Generamos predicciones en el conjunto de holdout_train
        preds = []
        for model in self.models:
            preds.append(model.predict(X_holdout_train))
        
        # Creamos un nuevo dataset con las predicciones de los modelos base
        X_meta = np.column_stack(preds)

        # Entrenamos el modelo meta en el conjunto de holdout_train
        self.meta_model.fit(X_meta, y_holdout_train)

        return training_time

    def predict(self, X):
        """
        Predice en modo blending
        """

        preds = []
        for model in self.models:
            preds.append(model.predict(X))

        X_meta = np.column_stack(preds)

        return self.meta_model.predict(X_meta)

    def predict_proba(self, X):
        """
        Predice en modo blending
        """

        preds = []
        for model in self.models:
            preds.append(model.predict(X))

        X_meta = np.column_stack(preds)

        return self.meta_model.predict_proba(X_meta)


In [108]:
svm1_model = SVC(
                    C=0.1,
                    gamma='scale',
                    kernel='linear',
                    probability=True,
                    class_weight='balanced',
                    random_state=42
                )
svm2_model = SVC(
                    C=1,
                    gamma='scale',
                    kernel='linear',
                    probability=True,
                    class_weight='balanced',
                    random_state=42
                )
svm3_model = SVC(
                    C=10,
                    gamma='scale',
                    kernel='linear',
                    probability=True,
                    class_weight='balanced',
                    random_state=42
                )


weak_learners = [svm1_model, svm2_model, svm3_model]

meta_model = SVC(
                    C=1,
                    gamma='scale',
                    kernel='linear',
                    probability=True,
                    class_weight='balanced',
                    random_state=42
                )

ensemble2 = BlendingClassifier(weak_learners, meta_model)

ensemble2_training_time = ensemble2.fit(X_train, y_train['depression'])

y_pred_ensemble2 = ensemble2.predict(X_val)
y_prob_ensemble2 = ensemble2.predict_proba(X_val)[:, 1]
print_classification_report(y_val['stress'], y_pred_ensemble2, y_prob_ensemble2, label="Ensemble 2")
print("Elapsed trainint time: ", ensemble2_training_time, "s")


Ensemble 2 Classification Report:
              precision    recall  f1-score   support

       False       0.00      0.00      0.00       315
        True       0.43      1.00      0.60       241

    accuracy                           0.43       556
   macro avg       0.22      0.50      0.30       556
weighted avg       0.19      0.43      0.26       556

Ensemble 2 ROC AUC: 0.4821
Ensemble 2 Precision-Recall AUC: 0.4294
Elapsed trainint time:  0.03613901138305664 s


### 1.3 Ensamble 3: Heterogéneo usando Blending de SVM, XGBoost, LightGBM

In [109]:
weak_learners = [xgb_model, svm_model, lgbm_model]

ensemble3 = BlendingClassifier(weak_learners, XGBClassifier())

ensemble3_training_time = ensemble3.fit(X_train, y_train['depression'])

y_pred_ensemble3 = ensemble3.predict(X_val)
y_prob_ensemble3 = ensemble3.predict_proba(X_val)[:, 1]
print_classification_report(y_val['stress'], y_pred_ensemble3, y_prob_ensemble3, label="Ensemble 3")
print("Elapsed trainint time: ", ensemble3_training_time, "s")


Ensemble 3 Classification Report:
              precision    recall  f1-score   support

       False       0.56      0.95      0.70       315
        True       0.21      0.02      0.03       241

    accuracy                           0.55       556
   macro avg       0.38      0.48      0.37       556
weighted avg       0.41      0.55      0.41       556

Ensemble 3 ROC AUC: 0.4614
Ensemble 3 Precision-Recall AUC: 0.3859
Elapsed trainint time:  0.17583394050598145 s


### 1.4 Ensamble 4: Heterogéneo usando Stacking de SVM, XGBoost, LightGBM


El método de stacking consiste en apilar la salida de cada estimador individual y utilizar un clasificador para calcular la predicción final. El apilamiento permite aprovechar la fortaleza de cada estimador individual al usar su salida como entrada para un estimador final.

In [110]:
estimators = [
    ("SVM", svm_model),
    ("XGBoost", xgb_model),
    ("LightGBM", lgbm_model),
]

ensemble4 = StackingClassifier(estimators=estimators, final_estimator=XGBClassifier())
ensemble4

StackingClassifier(estimators=[('SVM',
                                SVC(C=10, class_weight='balanced',
                                    kernel='linear', probability=True,
                                    random_state=42)),
                               ('XGBoost',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None,
                                              feature_typ...
                                                 grow_policy=None,
                                                 importance_type=None,
                                                 interaction_constraints=None,
                                                 learning_rate=None,
                                                 max_bin=None,
                                                 max_cat_threshold=None,
                                                 max_cat_to_onehot=None,
                                                 max_delta_step=None,
                                                 max_depth=None,
                                                 max_leaves=None,
                                                 min_child_weight=None,
                                                 missing=nan,
                                                 monotone_constraints=None,
                                                 multi_strategy=None,
                                                 n_estimators=None, n_jobs=None,
                                                 num_parallel_tree=None,
                                                 random_state=None, ...))

In [111]:
start_time = time.time()
ensemble4.fit(X_train, y_train['depression'])
ensemble4_training_time = time.time() - start_time

y_pred_ensemble4 = ensemble4.predict(X_val)
y_prob_ensemble4 = ensemble4.predict_proba(X_val)[:, 1]
print_classification_report(y_val['stress'], y_pred_ensemble4, y_prob_ensemble4, label="Ensemble 4")
print("Elapsed trainint time: ", ensemble4_training_time, "s")

Ensemble 4 Classification Report:
              precision    recall  f1-score   support

       False       0.57      0.85      0.69       315
        True       0.48      0.17      0.26       241

    accuracy                           0.56       556
   macro avg       0.53      0.51      0.47       556
weighted avg       0.53      0.56      0.50       556

Ensemble 4 ROC AUC: 0.5023
Ensemble 4 Precision-Recall AUC: 0.4830
Elapsed trainint time:  0.8573448657989502 s


# Tabla comparativa

Una vez que se han generado los modelos de ensamble, sintetizamos a continuación los resultados en una tabla comparativa en la que se incluyen los modelos individuales de la fase previa. Los modelos se ordenan por la métrica principal (curva ROC), pero el resumen incorpora adicionalmente:

[TODO: agregar las otras dos métricas: precision, accuracy]

Se incluyen los tiempos de entrenamiento y se elige el modelo final alineado con los objetivos y necesidades del negocio, que en este caso es disminuir el número de Falsos Positivos (optimizar precision), es decir disminuir el número de casos donde se afirma que alguien tiene el padecimiento cuando en realidad no lo tiene. Esto es importante en el caso específico del negocio que estamos trabajando pues el algoritmo se utilizará en la sociedad en general no en aplicaciones de alta seguridad como podría ser para evaluación de pilotos de avión.


# Gráficos significativos

Del modelo elegido, se generan algunos gráficos significativos con su interpretación:
- Curva ROC
- Matriz de confusión
- Curva de Precisión-recall
- Tendencia y predicción (para series temporales)